## Europe Happiness ##

Anticipation of happiness, Oil painting on canvas, by Leonid Afremov:

In [1]:
from IPython.display import Image
Image(url='https://afremov.com/image.php?type=P&id=18696')

In this notebook we visualize Europe country happiness scores. Data are retrieved  from [World Happiness Report 2016](http://worldhappiness.report/wp-content/uploads/sites/2/2016/03/HR-V1_web.pdf).


In [25]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm

In [26]:
df=pd.read_excel('Europe-Happiness.xlsx')
df

,Country,World-Rank,Score
0,Denmark,1,7.526
1,Switzerland,2,7.509
2,Iceland,3,7.501
3,Norway,4,7.498
4,Finland,5,7.413
5,Netherlands,7,7.339
6,Sweden,10,7.291
7,Israel,11,7.267
8,Austria,12,7.119
9,Germany,16,6.994


In [27]:
score=df['Score'].values
country=list(df.Country)
country[13]='U Kingdom'
country[14]='Czech Rep'
country[38]='Bosn Herzeg'
world_rk=list(df['World-Rank'])

In [28]:
import plotly.plotly as py
from plotly.graph_objs import *

Function that maps a normalized happiness score to a color in the chosen colormap:

In [29]:
def map_z2color(val, colormap, vmin, vmax):
    #map the normalized value val to a corresponding color in the mpl colormap
    
    if vmin>=vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(val-vmin)/float((vmax-vmin))#normalize val
    C=map(np.uint8, np.array(colormap(t)[:3])*255)
    #convert to a Plotly color code:
    return 'rgb'+str((C[0], C[1], C[2]))
       
    

In [30]:
def set_layout(title, plot_size):# set plot layout
    axis=dict(showline=False, 
              zeroline=False,
              showgrid=False,
              showticklabels=False,
              title='' 
             )

    return Layout(title=title,
                  font=Font(size=12), 
                  xaxis=XAxis(axis, range=[-3.5, 4]),
                  yaxis=YAxis(axis, range=[-4.5, 3.5]),
                  showlegend=False,
                  width=plot_size,
                  height=plot_size,
                  hovermode='closest',
                  annotations=[]        
                 )  

In [31]:
def set_annotation(x, y, anno_text,  angle, fontsize=11): # annotations
    return Annotation(x= x,  
                      y= y,       
                      text= anno_text,      
                      textangle=angle, # angle in degrees 
                      font= Font(size=fontsize),  
                      showarrow=False     
                     ) 

We visualize Europe country happiness through a circular barchart. The height of each bar is proportional to the corresponding happiness score:

In [32]:
bar_height=[score[k]/4 for k in range(49)]

The bars are inserted along the unit circle, starting with the angular position $\pi/2$, in the anti-clockwise direction: 

In [33]:
theta=[np.pi/2+(2*k+1)*np.pi/72  for k in range(49)]# angular position of base bar centers

In [34]:
xi=[np.cos(theta[k]) for k in range(49)]# starting bar position
yi=[np.sin(theta[k]) for k in range(49)]


xf=[(bar_height[k]+1)*np.cos(theta[k]) for k in range(49)]#end bar position
yf=[(bar_height[k]+1)*np.sin(theta[k]) for k in range(49)]

xm=[(xi[k]+xf[k])*0.5   for k in range(49)]#mid bar position for inserting hovering text
ym=[(yi[k]+yf[k])*0.5   for k in range(49)]

xpos_t=[(bar_height[k]+1.32)*np.cos(theta[k]) for k in range(49)]#text position
ypos_t=[(bar_height[k]+1.32)*np.sin(theta[k]) for k in range(49)]

The decreased level of happiness is illustrated through  the Viridis colormap:

In [35]:
import matplotlib.cm as cm
cmap=cm.viridis

In [36]:
vmin=score[-1]
vmax=score[0]
bar_colors=[map_z2color(score[k], cmap, vmin, vmax) for k in range(49)]

Define the hover text:

In [61]:
text=[country[k]+'<br>Score: '+'{:0.3f}'.format(score[k])+'<br>World rank: ' 
      +'{:d}'.format(world_rk[k])  for  k in range(len(score))]

Set position of the hover text inside bars:

In [62]:
trace=Scatter(x=xm,
              y=ym,
              name='',
              mode='markers' ,
              marker=dict(size=0.05, color=bar_colors[15]),
              text=text,
              hoverinfo='text')
tracet=Scatter(x=xf,
               y=yf,
               name='',
               mode='markers' ,
               marker=dict(size=0.05, color=bar_colors[15]), 
               text=text,
               hoverinfo='text')

In [63]:
traceB=[Scatter(x=[xi[k], xf[k], None], # Circular bars are drawn as lines of width 9
                y=[yi[k], yf[k], None], 
                mode='lines', 
                line=dict(color=bar_colors[k], width=9),
                hoverinfo='none')  
        for k in range(49)]


In [64]:
title="Europe Happiness Score and Global Ranking"+\
"<br>Data: World Happiness Report, 2016"+\
"<a href='http://worldhappiness.report/wp-content/uploads/sites/2/2016/03/HR-V1_web.pdf'> [1]</a>"
layout=set_layout('', 1200)

In [65]:
rot_angle=[87.5, 82.5, 77.5, 72.5, 67.5, 62.5, 57.5, 52.5, 47.5, 42.5, 37.5, 32.5, 27.5, 22.5, 17.5,
         12.5, 7.5, 2.5, -2.5, -7.5, -12.5, -17.5, -22.5, -27.5, -32.5, -37.5, -42.5, -47.5, 
         -52.5, -57.5, -62.5, -67.5, -72.5, -77.5, -82.5, -87.5, 87.5, 82.5, 77.5, 72.5, 67.5,
         62.5, 57.5, 52.5, 47.5, 42.5, 37.5, 32.5, 27.5]# angles for anno_text position

Insert annotations:

In [66]:
for k in range(49):
    layout['annotations']+=[set_annotation(xpos_t[k], ypos_t[k], country[k],  rot_angle[k],  fontsize=10)]
layout['annotations']+=[set_annotation(0.5, 0.1, title, 0 ) ]    

In [67]:
fig=Figure(data=Data(traceB+[trace, tracet]), layout=layout)
py.sign_in('empet', 'my_api_key')
py.plot(fig, filename='Europe-Happiness')

u'https://plot.ly/~empet/13870'

[https://plot.ly/~empet/13870/vs/](https://plot.ly/~empet/13870/vs/)

In [39]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()
